# This notebook will correct WCS for all kcwi notebooks

## Improves the coadd with Drizzle

## For south pointing, we fit a 2D Gaussian to QSO to improve WCS

## For North Pointing, we align continuum source (foreground galaxy) with it's proper position

In [1]:
# Necessary imports

import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from scipy.optimize import curve_fit
import scipy.optimize as opt
from tqdm import tqdm
from scipy import signal
from scipy.constants import c
from scipy.ndimage import gaussian_filter
from scipy import ndimage, misc
from astropy.modeling import models, fitting
from astropy.wcs import WCS
import time


In [2]:
def correct_WCS(number,RA,DEC,X,Y):
    hdul = fits.open('trimcube/kb220131_'+number+'_tr_icubes.fits')
    DATA = hdul[0].data # data string
    header = hdul[0].header
    w, ra, dec = np.shape(hdul[0].data)
    first_lamb = header['CRVAL3']
    spacing = header['CD3_3']
    Lambda = np.arange(first_lamb, first_lamb + spacing*w, spacing)
    hdul.close()
    whitelight = np.mean(DATA,0)
    whitelight /= np.sum(whitelight)
    y, x = np.shape(whitelight)
    x,y = np.meshgrid(np.arange(0, x, 1), np.arange(0, y, 1))
    # initial guess
    psf_guess = models.Gaussian2D(amplitude=1,x_mean=X,y_mean=Y,x_stddev=1,y_stddev=3,theta=0)
    fitter = fitting.LevMarLSQFitter()
    fit = fitter(psf_guess, x, y, whitelight)
    QSOX = fit.x_mean.value 
    QSOY = fit.y_mean.value
    REAL_RA = RA
    REAL_DEC = DEC
    
    hdul = fits.open('trimcube/kb220131_'+number+'_tr_icubes.fits')
    header = hdul[0].header
    DATA = hdul[0].data
    hdul.close()

    header['CRVAL1'] = REAL_RA
    header['CRVAL2'] = REAL_DEC
    header['CRPIX1'] = QSOX
    header['CRPIX2'] = QSOY
    w = WCS(header)
    sky = w.pixel_to_world(14.5,50,1)
    RA = sky[0].ra.deg
    DEC = sky[0].dec.deg

    sky = w.pixel_to_world(QSOX,QSOY,1)
    RA_qso = sky[0].ra.deg
    DEC_qso = sky[0].dec.deg

    diffra = RA_qso - REAL_RA
    diffdec = DEC_qso - REAL_DEC
    header['CRVAL1'] = RA - diffra
    header['CRVAL2'] = DEC - diffdec 
    header['CRPIX1'] = 14.5
    header['CRPIX2'] = 50
    hdu = fits.PrimaryHDU(data=DATA)
    hdu.header = header
    hdu.writeto('trimcube/kb220131_'+number+'_tr_icubes.fits',overwrite=True)

    hdul = fits.open('trimcube/kb220131_'+number+'_tr_vcubes.fits')
    header = hdul[0].header
    VAR = hdul[0].data
    hdul.close()
    header['CRVAL1'] = REAL_RA
    header['CRVAL2'] = REAL_DEC
    header['CRPIX1'] = QSOX
    header['CRPIX2'] = QSOY
    
    hdu = fits.PrimaryHDU(data=VAR)
    hdu.header = header
    hdu.writeto('trimcube/kb220131_'+number+'_tr_vcubes.fits',overwrite=True)

    print('Correct WCS on icubes/vcubes. Updated Fits files in "trimcube/" ')

                                   

In [3]:

# Foreground Star is present in both South and North Pointing
RA = 138.41036
DEC = -1.11549

# South
numbers = [53,54,55,56,57,58,65]
for n in numbers:
    number = str(n)
    correct_WCS(number,RA,DEC,21.5,61)

# North
numbers = [59,61,62,63,64,66]
for n in numbers:
    number = str(n)
    correct_WCS(number,RA,DEC,21,3)

Correct WCS on icubes/vcubes. Updated Fits files in "trimcube/" 
Correct WCS on icubes/vcubes. Updated Fits files in "trimcube/" 
Correct WCS on icubes/vcubes. Updated Fits files in "trimcube/" 
Correct WCS on icubes/vcubes. Updated Fits files in "trimcube/" 
Correct WCS on icubes/vcubes. Updated Fits files in "trimcube/" 
Correct WCS on icubes/vcubes. Updated Fits files in "trimcube/" 
Correct WCS on icubes/vcubes. Updated Fits files in "trimcube/" 
Correct WCS on icubes/vcubes. Updated Fits files in "trimcube/" 
Correct WCS on icubes/vcubes. Updated Fits files in "trimcube/" 
Correct WCS on icubes/vcubes. Updated Fits files in "trimcube/" 
Correct WCS on icubes/vcubes. Updated Fits files in "trimcube/" 
Correct WCS on icubes/vcubes. Updated Fits files in "trimcube/" 
Correct WCS on icubes/vcubes. Updated Fits files in "trimcube/" 
